In [90]:
#import statements
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation,MaxPooling2D,Dropout,Flatten
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from keras import optimizers
from sklearn.ensemble import RandomForestRegressor
import os

#df is data from csv file
filename_train_csv = "data/train_free_throws.csv"
df=pd.read_csv(filename_train_csv, header = None)

#change this csv for testing
filename_test_csv = "data/test_free_throws.csv"
mainDF=pd.read_csv(filename_test_csv, header = None)
datagen=ImageDataGenerator() #rescale=1./255)


df.columns = ['0','1']
mainDF.columns = ['0','1']


# main generator is the one for test data
main_generator=datagen.flow_from_dataframe(dataframe=mainDF, directory="data/", x_col="0",
                                           y_col="1", class_mode="raw", target_size=(320,180), batch_size=32)

valid_generator=datagen.flow_from_dataframe(dataframe=df, directory="data/", x_col="0",
                                           y_col="1", class_mode="raw", target_size=(320,180), batch_size=32)

Found 30000 validated image filenames.
Found 20000 validated image filenames.


In [92]:
model_kernel_size = (3,3)
input_shape=(320,180,3)
num_classes = 2  # True of False classification

model = Sequential()
model.add(Conv2D(6, kernel_size=model_kernel_size, activation='relu', input_shape= input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size= model_kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))

In [93]:
#model.compile(optimizers.rmsprop(lr=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizers.rmsprop(lr=0.0001),loss="sparse_categorical_crossentropy", metrics=["accuracy"])

STEP_SIZE_TRAIN = main_generator.n//main_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
hist = model.fit_generator(generator=main_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1)

#score = model.evalutate(x_test,y_test, verbose = 0)

# training values
acc = hist.history['acc']
loss_ = hist.history['loss']

print("test accuracy: " , acc)

Epoch 1/1
937/937 [==============================] - 76s 81ms/step - loss: 2.1027 - acc: 0.8695 - val_loss: 1.3346 - val_acc: 0.9172
('test accuracy: ', [0.8694970651013874])


NameError: name 'i_rgb_ht' is not defined

In [68]:
filename_train_csv = "data/free_throw_training_data.csv"
df=pd.read_csv(filename_train_csv, header = None)

cut = 30000
cut_end = cut + 20000
df1 = df.iloc[:cut, :]
df2 = df.iloc[cut:cut_end, :]
df1.shape, df2.shape

df1.to_csv("data/test_free_throws.csv", header= None, index=False)
df2.to_csv("data/train_free_throws.csv", header= None, index=False)

df1.shape, df2.shape

((30000, 2), (20000, 2))

In [94]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [100]:
from keras.models import model_from_json
#def inference(csv_file):
csv_file = 'data/test_free_throws.csv'
#load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load weights into new model
loaded_model.load_weights("model.h5")
#... read csv file
# df is data from csv file
filename = csv_file
df=pd.read_csv(filename, header = None)
df.columns = ['0','1']
valid_generator = datagen.flow_from_dataframe(dataframe=df, directory="data/", x_col="0",y_col="1",
                                            class_mode="raw", target_size=(320,180), batch_size=32)

#... use the model to do inference
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
score = model.evaluate_generator(generator = valid_generator, steps = STEP_SIZE_VALID,
                                 use_multiprocessing = False, verbose = 0 )

print('\n answer:\n')
print('Test loss:', score[0] )
print('Test accuracy:', score[1] )
print('\n')

#... compute accuracy,recall and precision score using 0.5 threshold
#return {'accuracy':accuracy, 'recall':recall, 'precision':precision}

Found 30000 validated image filenames.

 answer:

('Test loss:', 2.1023836470464503)
('Test accuracy:', 0.8695637673425827)




In [1]:
s3://asu-fox-hackathon-testing

SyntaxError: invalid syntax (<ipython-input-1-42d323b39775>, line 1)

In [2]:
!aws s3://asu-fox-hackathon-testing

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument command: Invalid choice, valid choices are:

acm                                      | acm-pca                                 
alexaforbusiness                         | amplify                                 
apigateway                               | apigatewaymanagementapi                 
apigatewayv2                             | application-autoscaling                 
application-insights                     | appmesh                                 
appstream                                | appsync                                 
athena                                   | autoscaling                             
autoscaling-plans                        | backup                                  
batch                                    | budgets                                 
ce  